In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from collections import Counter


In [2]:
fn = "Erklärung Gemeinden Jahr 2020 Tramin.xlsx"

In [3]:
df = pd.read_excel(fn)

In [4]:
    df.columns = [
        "Numero \nprogressivo",
        "Cognome e nome bambino",
        "Data di nascita",
        "Codice fiscale",
        "Assistente domiciliare all'infanzia",
        "Comune di residenza assistente domiciliare all'infanzia",
        "Data inizio contratto (o data inizio assistenza se diversa)",
        "Data fine contratto\n(o data fine assistenza se diversa) *",
        "Ore di assistenza \n ai sensi della delibera\nn. 666/2019",
        "Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020",
        "Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020",
        "Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)",
        "Ore totali rendicontate per il 2020",
    ]

In [5]:
traeger = df.iloc[2]

In [7]:
type(traeger[4]) == type("string")

False

In [8]:
traeger = traeger[3].title()
gemeinde = df.iloc[3]
gemeinde = gemeinde[3]
gemeinde

'Tramin'

In [9]:
df = df.drop(labels=range(0, 8), axis=0)
cond = df["Numero \nprogressivo"] == "*"
df.loc[cond, "Numero \nprogressivo"] = 99
df.loc[cond, "Cognome e nome bambino"] += " *"
df["Numero \nprogressivo"] = pd.to_numeric(
    df["Numero \nprogressivo"], errors="coerce"
)
validi = df["Numero \nprogressivo"].notna()
df = df[validi]

In [10]:
condizione = pd.isnull(
    df["Data fine contratto\n(o data fine assistenza se diversa) *"]
)
anno_riferimento = 2020
df.loc[
    condizione, "Data fine contratto\n(o data fine assistenza se diversa) *"
] = "12-31-" + str(anno_riferimento)

anno_riferimento = 2020
df.loc[
    condizione, "Data fine contratto\n(o data fine assistenza se diversa) *"
] = "12-31-" + str(anno_riferimento)
df["Data di nascita"] = df["Data di nascita"].astype("datetime64[ns]")

df["Data fine contratto\n(o data fine assistenza se diversa) *"] = df[
    "Data fine contratto\n(o data fine assistenza se diversa) *"
].astype("datetime64[ns]")

df["Data inizio contratto (o data inizio assistenza se diversa)"] = df[
    "Data inizio contratto (o data inizio assistenza se diversa)"
].astype("datetime64[ns]")

df["Numero \nprogressivo"] = pd.to_numeric(
    df["Numero \nprogressivo"], errors="coerce"
)

df = df.drop(["Numero \nprogressivo"], axis=1)

df.insert(1, "Comune", gemeinde)
df.insert(1, "Ente", traeger)
df = df.fillna(0)
df["Codice fiscale"] = df["Codice fiscale"].str.upper()

ar = 2020

df["inizioNorm"] = df[
    "Data inizio contratto (o data inizio assistenza se diversa)"
]

df["fineNorm"] = df["Data fine contratto\n(o data fine assistenza se diversa) *"]

iniz = df["inizioNorm"].dt.year < ar

df.loc[iniz, "inizioNorm"] = "01-01-" + str(
    ar
) 

fin = df["fineNorm"].dt.year > ar

df.loc[fin, "fineNorm"] = "12-31-" + str(ar)

df.insert(
    9, "GiorniAssistenzaAnnoRiferimento", 0
) 

df["GiorniAssistenzaAnnoRiferimento"] = (
    df["fineNorm"] - df["inizioNorm"]
).dt.days + 1

In [11]:
condizioneerrore = ((1920 * (df["GiorniAssistenzaAnnoRiferimento"])) / 366) < df[
    "Ore totali rendicontate per il 2020"
]
    
condizioneerrore2 = ((1920 * (df.groupby("Codice fiscale")["GiorniAssistenzaAnnoRiferimento"].transform("sum")) / 366)) < df.groupby("Codice fiscale")["Ore totali rendicontate per il 2020"].transform("sum")

In [29]:
for i in df[~codfisc1]["Codice fiscale"]:
    print(i)

KJNMIA18M43A952A
KJNMIA18M43A952A
TMGMRM19B41A952N
TMGMRM19B41A952N
TMGMRM19B41A952N
TMGMRM19B41A952N
VNCDLN19B19A952G
VNCDLN19B19A952G
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P


In [19]:
codfisc1 = df.groupby("Codice fiscale")["Codice fiscale"].transform("count") == 1

In [27]:
df[condizioneerrore2 & ~codfisc1]

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm
12,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Ambach Maria,Eppan,2020-08-17,2020-08-28,12,71.50,0.00,0,0.0,71.50,2020-08-17,2020-08-28
13,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Huber Melanie,Margreid,2020-11-25,2020-11-27,3,26.50,0.00,0,0.0,26.50,2020-11-25,2020-11-27
14,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-01-02,2020-05-06,126,189.75,270.50,0,332.5,792.75,2020-01-02,2020-05-06
15,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-07-20,2020-12-31,165,804.00,10.25,0,0.0,814.25,2020-07-20,2020-12-31
